In [11]:
import re
import os
from pathlib import Path

import pandas as pd
import numpy as np

from model.DRST import DRST

current_dir = os.getcwd()

In [2]:
# datasets = {
#     'Health Insurance': {
#         'path': 'datasets/health_insurance.csv',
#         'orginal_data': None,
#         'encoded_data': None,
#         'shape': [0, 0],
#         'columns_name': [],
#         'drst_data': None,
#         'drst_continous_columns': [],
#         'drst_interval_number': [],
#     }
# }
datasets = ['datasets/bands.csv', 'datasets/crx.csv', 'datasets/echocardiogram.csv']

In [15]:

def setup_data(path):
    dataset_name = str.split(path, '/')[1].split('.')[0]
    Path('%s\\output\\datasets\\%s' % (current_dir, dataset_name)).mkdir(parents=True,exist_ok=True)
    Path('%s\\output\\datasets\\%s\\info.txt' % (current_dir, dataset_name)).touch(exist_ok=True)
    info_file = open('%s\\output\\datasets\\%s\\info.txt' % (current_dir, dataset_name), 'a+')
    print('%s ====> LOADED' % dataset_name)
    data = pd.read_csv(path, header=None)
    data = data.replace('?', np.NaN)
    data.dropna(thresh=(data.shape[0] * .93), axis=1, inplace=True)
    data.dropna(thresh=(data.shape[1] * .93), axis=0, inplace=True)
    data.fillna(data.mode().iloc[0], inplace=True)
    data.columns = [str(c) for c in data.columns]
    data.to_csv('%s\\output\\datasets\\%s\\%s-cleaned.csv' % (current_dir, dataset_name, dataset_name))
    info_file.write('Number of objects = %s\n' % data.shape[0])
    info_file.write('Number of features = %s\n' % data.shape[1])

    
    info_file.close()
for path in datasets:
    setup_data(path=path)

bands ====> LOADED
crx ====> LOADED
echocardiogram ====> LOADED


In [139]:
drst = DRST(comb_max_depth=1, decision_column_name=data.columns[-1], topN_thrshold=.7, save_output=False)
drst_fit, continuous_columns = drst.fit(data, ensamble_threshold=0.9)
drst_fit.drop(continuous_columns, axis=1, inplace=True)

['1', '3', '7', '9']
1    3
3    8
7    3
9    3
dtype: int64


,0,2,4,5,6,8,10,1_AFTER,3_AFTER,7_AFTER,9_AFTER
0,0,1,1,1,0,35,0,1,3,0,0
1,0,1,0,2,1,15,1,2,3,2,0
2,0,1,0,0,1,29,0,2,6,0,0
3,0,1,1,0,0,28,0,2,3,0,0
4,0,1,0,1,1,42,0,1,7,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,0,1,1,35,0,1,3,0,2
996,1,1,0,1,1,35,1,1,4,0,2
997,1,1,0,1,1,39,1,1,3,1,2
998,0,1,0,2,1,15,0,0,6,0,2
